In [19]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import pandas as pd

2.8.2


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Softmax,Conv2D,MaxPool2D,AvgPool2D

In [3]:
x_train=np.load("/content/drive/MyDrive/MLSP Classifier/Single timeframe Final Dataset /x_train.npy")
x_validation=np.load("/content/drive/MyDrive/MLSP Classifier/Single timeframe Final Dataset /x_validation.npy")
x_test=np.load("/content/drive/MyDrive/MLSP Classifier/Single timeframe Final Dataset /x_test.npy")
y_train=np.load("/content/drive/MyDrive/MLSP Classifier/Single timeframe Final Dataset /y_train.npy")
y_validation=np.load("/content/drive/MyDrive/MLSP Classifier/Single timeframe Final Dataset /y_validation.npy")
y_test=np.load("/content/drive/MyDrive/MLSP Classifier/Single timeframe Final Dataset /y_test.npy")

In [4]:
x_train.shape

(750, 1, 128)

In [5]:
x_train_new=np.expand_dims(x_train,axis=3)
x_validation_new=np.expand_dims(x_validation,axis=3)
x_test_new=np.expand_dims(x_test,axis=3)

In [6]:
x_train_new.shape

(750, 1, 128, 1)

In [7]:
abc=Conv2D(filters=10,kernel_size=(5,1),padding='same')(x_train_new[0:4])

In [8]:
abc.shape

TensorShape([4, 1, 128, 10])

In [9]:
abd=MaxPool2D(pool_size=(1,4))(abc)

In [10]:
abd.shape

TensorShape([4, 1, 32, 10])

In [11]:
model=Sequential([
    Conv2D(64,(5,1),activation='relu',input_shape=(1,128,1),padding='same'),
    MaxPool2D(pool_size=(1,4)),
    Conv2D(32,(5,1),activation='relu',padding='same'),
    MaxPool2D(pool_size=(1,2)),
    Conv2D(16,(5,1),activation='relu' ,padding='same'),
    MaxPool2D(pool_size=(1,2)),
    Conv2D(8,(5,1),activation='relu' ,padding='same'),
    AvgPool2D(pool_size=(1,2)),
    Flatten(),
    Dense(512,activation='tanh'),
    Dense(256,activation='tanh'),
    Dense(128,activation='tanh'),
    Dense(64,activation='relu'),
    Dense(10,activation='softmax')
])

In [12]:
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

In [13]:
model.fit(x_train_new,y_train,batch_size=16,epochs=500,validation_data=(x_validation_new,y_validation))

Epoch 1/500
47/47 [==============================] - 3s 15ms/step - loss: 2.3014 - accuracy: 0.1027 - val_loss: 2.3011 - val_accuracy: 0.1040
Epoch 2/500
47/47 [==============================] - 0s 5ms/step - loss: 2.2908 - accuracy: 0.1427 - val_loss: 2.2822 - val_accuracy: 0.1120
Epoch 3/500
47/47 [==============================] - 0s 5ms/step - loss: 2.2506 - accuracy: 0.1693 - val_loss: 2.2239 - val_accuracy: 0.1520
Epoch 4/500
47/47 [==============================] - 0s 6ms/step - loss: 2.1866 - accuracy: 0.1920 - val_loss: 2.1707 - val_accuracy: 0.1600
Epoch 5/500
47/47 [==============================] - 0s 6ms/step - loss: 2.1460 - accuracy: 0.2013 - val_loss: 2.1405 - val_accuracy: 0.1840
Epoch 6/500
47/47 [==============================] - 0s 5ms/step - loss: 2.1233 - accuracy: 0.2373 - val_loss: 2.1230 - val_accuracy: 0.1760
Epoch 7/500
47/47 [==============================] - 0s 5ms/step - loss: 2.1063 - accuracy: 0.2347 - val_loss: 2.1030 - val_accuracy: 0.1440
Epoch 8/500


In [14]:
x_test_final=np.load("/content/drive/MyDrive/MLSP Classifier/Audio_Classification-MLSP-test/x_test_final.npy")

In [15]:
l=model.predict(x_test_final)

In [16]:
class_dictionary={0: 'Bark', 1: 'Meow', 2: 'Siren', 3: 'Shatter', 4: 'Knock', 5: 'Crying_and_sobbing', 6: 'Microwave_oven', 7: 'Vehicle_horn_and_car_horn_and_honking', 8: 'Doorbell', 9: 'Walk_and_footsteps'}

In [17]:
tags=[]
for i in range(l.shape[0]):
 tags.append(class_dictionary[np.argmax(l[i])])

In [20]:
data_with_labels=pd.read_csv("/content/drive/MyDrive/MLSP Classifier/Audio_Classification-MLSP-test/final.csv")
filename=data_with_labels['Filename']

In [24]:
final=pd.DataFrame({
    'fileid':filename,
    'prediction':tags
})

In [25]:
final.to_csv('/content/drive/MyDrive/MLSP Classifier/CNN_model_prediction.csv')